[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/Retrieval-based-Voice-Conversion-WebUI-colab/blob/main/retrieval_based_voice_conversion_webui_colab.ipynb)

In [ ]:
# https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI/blob/main/Retrieval_based_Voice_Conversion_WebUI.ipynb modified
# https://colab.research.google.com/drive/1r4IRL0UA7JEoZ0ZK8PKfMyTIBHKpyhcw?usp=sharing modified

# Model Tracker https://docs.google.com/spreadsheets/d/1qzeFdpUPr7E0jOFwWSXd8LF30ZLjz1CSVEBiG8gPHTU

%cd /content
%env TF_CPP_MIN_LOG_LEVEL=1
!apt -y install -qq aria2
!pip install -q faiss-gpu fairseq gradio ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2 mega.py gdown

!git clone -b v1.0 https://github.com/camenduru/Retrieval-based-Voice-Conversion-WebUI
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!wget https://raw.githubusercontent.com/camenduru/EVC/main/easy-infer.py
!mkdir -p pretrained uvr5_weights

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o hubert_base.pt

import os
from mega import Mega
MODEL = 'BENEE'
MODELEPOCH = ''
url='https://drive.google.com/file/d/1nuUVTqg1_lja7JFctjd18W9hQ7rk7aDZ/view?usp=share_link'
MODELZIP = MODEL + '.zip'
modelname_path='/content/zips/'+MODELZIP
!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODEL}
if url != '':
  !mkdir -p /content/zips/
  if "drive.google.com" in url:
    !gdown $url --fuzzy -O $modelname_path
  elif "mega.nz" in url:
    m = Mega()
    m.download_url(url, '/content/zips')
  else:
    !wget $url -O /content/zips/{MODELZIP}
  !unzip -d /content/unzips/ -B /content/zips/{MODEL}.zip
else:
  raise Exception('The URL cannot be empty.')
for root, dirs, files in os.walk('/content/unzips'):
  for file in files:
    if "G_" in file:
        MODELEPOCH = file.split("G_")[1].split(".")[0]
  if MODELEPOCH == '':
    MODELEPOCH = '404'
  for file in files:
    file_path = os.path.join(root, file)
    if file.endswith(".npy") or file.endswith(".index"):
      !mv {file_path} /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODEL}/
    elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
      !mv {file_path} /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODEL}.pth
!rm -r /content/unzips/
!rm -r /content/zips/

%cd /content/Retrieval-based-Voice-Conversion-WebUI
!python3 easy-infer.py --colab --pycmd python3